<a href="https://colab.research.google.com/github/Rocklucca32/Facul/blob/main/Projeto_Integrado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sqlite3

# Banco de dados

class BancoDados:
    def __init__(self, nome_db="biblioteca.db"):
        self.conn = sqlite3.connect(nome_db)
        self.create_tables()

    def create_tables(self):
        cursor = self.conn.cursor()
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS livros (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            titulo TEXT,
            autor TEXT,
            ano INTEGER,
            copias INTEGER,
            emprestados INTEGER DEFAULT 0
        )
        """)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS usuarios (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            nome TEXT,
            contato TEXT
        )
        """)
        cursor.execute("""
        CREATE TABLE IF NOT EXISTS emprestimos (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            id_usuario INTEGER,
            id_livro INTEGER,
            FOREIGN KEY(id_usuario) REFERENCES usuarios(id),
            FOREIGN KEY(id_livro) REFERENCES livros(id)
        )
        """)
        self.conn.commit()

    def fechar(self):
        self.conn.close()

# Classes

class Livro:
    def __init__(self, id, titulo, autor, ano, copias, emprestados=0):
        self.id = id
        self.titulo = titulo
        self.autor = autor
        self.ano = ano
        self.copias = copias
        self.emprestados = emprestados

    def __str__(self):
        disponiveis = self.copias - self.emprestados
        return f"[ID: {self.id}] {self.titulo} - {self.autor} ({self.ano}) | Disponíveis: {disponiveis}"

    def esta_disponivel(self):
        return self.emprestados < self.copias


class Usuario:
    def __init__(self, id, nome, contato):
        self.id = id
        self.nome = nome
        self.contato = contato

    def __str__(self):
        return f"[ID: {self.id}] {self.nome} - Contato: {self.contato}"


class Biblioteca:
    def __init__(self, banco):
        self.banco = banco

    # Livros

    def cadastrar_livro(self, titulo, autor, ano, copias):
        cursor = self.banco.conn.cursor()
        cursor.execute(
            "INSERT INTO livros (titulo, autor, ano, copias) VALUES (?, ?, ?, ?)",
            (titulo, autor, ano, copias),
        )
        self.banco.conn.commit()
        print(f"Livro '{titulo}' cadastrado com ID {cursor.lastrowid}")

    def buscar_livro(self, termo):
        cursor = self.banco.conn.cursor()
        termo_wildcard = f"%{termo}%"
        cursor.execute("""
            SELECT id, titulo, autor, ano, copias, emprestados
            FROM livros
            WHERE titulo LIKE ? OR autor LIKE ? OR CAST(ano AS TEXT) LIKE ?
        """, (termo_wildcard, termo_wildcard, termo_wildcard))
        livros = cursor.fetchall()
        return [Livro(*l) for l in livros]

    def buscar_livro_por_id(self, id_livro):
        cursor = self.banco.conn.cursor()
        cursor.execute("SELECT id, titulo, autor, ano, copias, emprestados FROM livros WHERE id=?", (id_livro,))
        linha = cursor.fetchone()
        if linha:
            return Livro(*linha)
        return None

    def atualizar_emprestados(self, id_livro, delta):
        cursor = self.banco.conn.cursor()
        cursor.execute("UPDATE livros SET emprestados = emprestados + ? WHERE id=?", (delta, id_livro))
        self.banco.conn.commit()

    # Usuários

    def cadastrar_usuario(self, nome, contato):
        cursor = self.banco.conn.cursor()
        cursor.execute("INSERT INTO usuarios (nome, contato) VALUES (?, ?)", (nome, contato))
        self.banco.conn.commit()
        print(f"Usuário '{nome}' cadastrado com ID {cursor.lastrowid}")

    def buscar_usuario_por_id(self, id_usuario):
        cursor = self.banco.conn.cursor()
        cursor.execute("SELECT id, nome, contato FROM usuarios WHERE id=?", (id_usuario,))
        linha = cursor.fetchone()
        if linha:
            return Usuario(*linha)
        return None

    def listar_usuarios(self):
        cursor = self.banco.conn.cursor()
        cursor.execute("SELECT id, nome, contato FROM usuarios")
        return [Usuario(*u) for u in cursor.fetchall()]

    # Empréstimos

    def emprestar_livro(self, id_usuario, id_livro):
        usuario = self.buscar_usuario_por_id(id_usuario)
        if not usuario:
            print("Usuário não encontrado.")
            return

        livro = self.buscar_livro_por_id(id_livro)
        if not livro:
            print("Livro não encontrado.")
            return

        if not livro.esta_disponivel():
            print("Livro indisponível no momento.")
            return

        # Registrar empréstimo
        cursor = self.banco.conn.cursor()
        cursor.execute("INSERT INTO emprestimos (id_usuario, id_livro) VALUES (?, ?)", (id_usuario, id_livro))
        self.atualizar_emprestados(id_livro, 1)
        self.banco.conn.commit()
        print(f"Livro '{livro.titulo}' emprestado para {usuario.nome}.")

    def devolver_livro(self, id_usuario, id_livro):
        cursor = self.banco.conn.cursor()

        cursor.execute("""
            SELECT id FROM emprestimos
            WHERE id_usuario=? AND id_livro=?
            LIMIT 1
        """, (id_usuario, id_livro))
        linha = cursor.fetchone()

        if linha:
            id_emprestimo = linha[0]
            cursor.execute("DELETE FROM emprestimos WHERE id=?", (id_emprestimo,))
            self.atualizar_emprestados(id_livro, -1)
            self.banco.conn.commit()
            livro = self.buscar_livro_por_id(id_livro)
            usuario = self.buscar_usuario_por_id(id_usuario)
            print(f"Livro '{livro.titulo}' devolvido por {usuario.nome}.")
        else:
            print("Este livro não foi emprestado por este usuário.")

    # Relatórios

    def relatorio_livros_disponiveis(self):
        cursor = self.banco.conn.cursor()
        cursor.execute("""
            SELECT id, titulo, autor, ano, copias, emprestados
            FROM livros
            WHERE emprestados < copias
        """)
        livros = cursor.fetchall()
        print("\n--- Livros Disponíveis ---")
        for l in livros:
            print(Livro(*l))

    def relatorio_livros_emprestados(self):
        cursor = self.banco.conn.cursor()
        cursor.execute("""
            SELECT id, titulo, autor, ano, copias, emprestados
            FROM livros
            WHERE emprestados > 0
        """)
        livros = cursor.fetchall()
        print("\n--- Livros Emprestados ---")
        for l in livros:
            livro = Livro(*l)
            print(f"{livro} - Emprestados: {livro.emprestados}")

    def relatorio_usuarios(self):
        usuarios = self.listar_usuarios()
        print("\n--- Usuários Cadastrados ---")
        for u in usuarios:
            print(u)


# Interface

def menu():
    print("""
--- Biblioteca Digital ---
1. Cadastrar Livro
2. Cadastrar Usuário
3. Emprestar Livro
4. Devolver Livro
5. Consultar Livros
6. Relatórios
7. Sair
""")


def menu_relatorios(biblioteca):
    while True:
        print("""
--- Relatórios ---
1. Livros Disponíveis
2. Livros Emprestados
3. Usuários
4. Voltar
""")
        opcao = input("Escolha uma opção: ")
        if opcao == '1':
            biblioteca.relatorio_livros_disponiveis()
        elif opcao == '2':
            biblioteca.relatorio_livros_emprestados()
        elif opcao == '3':
            biblioteca.relatorio_usuarios()
        elif opcao == '4':
            break
        else:
            print("Opção inválida.")


def main():
    banco = BancoDados()
    biblioteca = Biblioteca(banco)

    while True:
        menu()
        opcao = input("Escolha uma opção: ")
        if opcao == '1':
            titulo = input("Título: ")
            autor = input("Autor: ")
            try:
                ano = int(input("Ano: "))
                copias = int(input("Número de cópias: "))
                biblioteca.cadastrar_livro(titulo, autor, ano, copias)
            except ValueError:
                print("Ano e número de cópias devem ser números.")
        elif opcao == '2':
            nome = input("Nome: ")
            contato = input("Contato: ")
            biblioteca.cadastrar_usuario(nome, contato)
        elif opcao == '3':
            try:
                id_usuario = int(input("ID do usuário: "))
                id_livro = int(input("ID do livro: "))
                biblioteca.emprestar_livro(id_usuario, id_livro)
            except ValueError:
                print("IDs inválidos. Use apenas números.")
        elif opcao == '4':
            try:
                id_usuario = int(input("ID do usuário: "))
                id_livro = int(input("ID do livro: "))
                biblioteca.devolver_livro(id_usuario, id_livro)
            except ValueError:
                print("IDs inválidos. Use apenas números.")
        elif opcao == '5':
            termo = input("Digite título, autor ou ano: ")
            encontrados = biblioteca.buscar_livro(termo)
            print("\n--- Livros Encontrados ---")
            for livro in encontrados:
                print(livro)
        elif opcao == '6':
            menu_relatorios(biblioteca)
        elif opcao == '7':
            print("Saindo...")
            banco.fechar()
            break
        else:
            print("Opção inválida.")


if __name__ == "__main__":
    main()
